In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('tree').getOrCreate()

In [2]:
from pyspark.ml import Pipeline

from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, DecisionTreeClassifier

In [3]:
data = spark.read.csv('/FileStore/tables/College.csv', inferSchema=True, header=True)

In [4]:
data.printSchema()

root
-- School: string (nullable = true)
-- Private: string (nullable = true)
-- Apps: integer (nullable = true)
-- Accept: integer (nullable = true)
-- Enroll: integer (nullable = true)
-- Top10perc: integer (nullable = true)
-- Top25perc: integer (nullable = true)
-- F_Undergrad: integer (nullable = true)
-- P_Undergrad: integer (nullable = true)
-- Outstate: integer (nullable = true)
-- Room_Board: integer (nullable = true)
-- Books: integer (nullable = true)
-- Personal: integer (nullable = true)
-- PhD: integer (nullable = true)
-- Terminal: integer (nullable = true)
-- S_F_Ratio: double (nullable = true)
-- perc_alumni: integer (nullable = true)
-- Expend: integer (nullable = true)
-- Grad_Rate: integer (nullable = true)

In [5]:
data.head(10)

Out[8]: [Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60),
 Row(School='Adelphi University', Private='Yes', Apps=2186, Accept=1924, Enroll=512, Top10perc=16, Top25perc=29, F_Undergrad=2683, P_Undergrad=1227, Outstate=12280, Room_Board=6450, Books=750, Personal=1500, PhD=29, Terminal=30, S_F_Ratio=12.2, perc_alumni=16, Expend=10527, Grad_Rate=56),
 Row(School='Adrian College', Private='Yes', Apps=1428, Accept=1097, Enroll=336, Top10perc=22, Top25perc=50, F_Undergrad=1036, P_Undergrad=99, Outstate=11250, Room_Board=3750, Books=400, Personal=1165, PhD=53, Terminal=66, S_F_Ratio=12.9, perc_alumni=30, Expend=8735, Grad_Rate=54),
 Row(School='Agnes Scott College', Private='Yes', Apps=417, Accept=349, Enroll=137, Top10perc=60, Top25perc=89, F_Undergrad=510, P_Undergrad=63, Outstate=12960, Room_Board=5450, Books=450, Personal=875, PhD=92, Terminal=97, S_F_Ratio=7.7, perc_alumni=37, Expend=19016, Grad_Rate=59),
 Row(School='Alaska Pacific University', Private='Yes', Apps=193, Accept=146, Enroll=55, Top10perc=16, Top25perc=44, F_Undergrad=249, P_Undergrad=869, Outstate=7560, Room_Board=4120, Books=800, Personal=1500, PhD=76, Terminal=72, S_F_Ratio=11.9, perc_alumni=2, Expend=10922, Grad_Rate=15),
 Row(School='Albertson College', Private='Yes', Apps=587, Accept=479, Enroll=158, Top10perc=38, Top25perc=62, F_Undergrad=678, P_Undergrad=41, Outstate=13500, Room_Board=3335, Books=500, Personal=675, PhD=67, Terminal=73, S_F_Ratio=9.4, perc_alumni=11, Expend=9727, Grad_Rate=55),
 Row(School='Albertus Magnus College', Private='Yes', Apps=353, Accept=340, Enroll=103, Top10perc=17, Top25perc=45, F_Undergrad=416, P_Undergrad=230, Outstate=13290, Room_Board=5720, Books=500, Personal=1500, PhD=90, Terminal=93, S_F_Ratio=11.5, perc_alumni=26, Expend=8861, Grad_Rate=63),
 Row(School='Albion College', Private='Yes', Apps=1899, Accept=1720, Enroll=489, Top10perc=37, Top25perc=68, F_Undergrad=1594, P_Undergrad=32, Outstate=13868, Room_Board=4826, Books=450, Personal=850, PhD=89, Terminal=100, S_F_Ratio=13.7, perc_alumni=37, Expend=11487, Grad_Rate=73),
 Row(School='Albright College', Private='Yes', Apps=1038, Accept=839, Enroll=227, Top10perc=30, Top25perc=63, F_Undergrad=973, P_Undergrad=306, Outstate=15595, Room_Board=4400, Books=300, Personal=500, PhD=79, Terminal=84, S_F_Ratio=11.3, perc_alumni=23, Expend=11644, Grad_Rate=80),
 Row(School='Alderson-Broaddus College', Private='Yes', Apps=582, Accept=498, Enroll=172, Top10perc=21, Top25perc=44, F_Undergrad=799, P_Undergrad=78, Outstate=10468, Room_Board=3380, Books=660, Personal=1800, PhD=40, Terminal=41, S_F_Ratio=11.5, perc_alumni=15, Expend=8991, Grad_Rate=52)]

In [6]:
from pyspark.ml.feature import VectorAssembler

In [7]:
data.columns

Out[10]: ['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [8]:
assembler = VectorAssembler(inputCols=['Apps',
                                       'Accept',
                                       'Enroll',
                                       'Top10perc',
                                       'Top25perc',
                                       'F_Undergrad',
                                       'P_Undergrad',
                                       'Outstate',
                                       'Room_Board',
                                       'Books',
                                       'Personal',
                                       'PhD',
                                       'Terminal',
                                       'S_F_Ratio',
                                       'perc_alumni',
                                       'Expend',
                                       'Grad_Rate'], 
                            outputCol='features')

In [9]:
output = assembler.transform(data)

In [10]:
from pyspark.ml.feature import StringIndexer

In [11]:
indexer = StringIndexer(inputCol='Private', outputCol='PrivateIndex')

In [12]:
output_fixed = indexer.fit(output).transform(output)

In [13]:
output_fixed.printSchema()

root
-- School: string (nullable = true)
-- Private: string (nullable = true)
-- Apps: integer (nullable = true)
-- Accept: integer (nullable = true)
-- Enroll: integer (nullable = true)
-- Top10perc: integer (nullable = true)
-- Top25perc: integer (nullable = true)
-- F_Undergrad: integer (nullable = true)
-- P_Undergrad: integer (nullable = true)
-- Outstate: integer (nullable = true)
-- Room_Board: integer (nullable = true)
-- Books: integer (nullable = true)
-- Personal: integer (nullable = true)
-- PhD: integer (nullable = true)
-- Terminal: integer (nullable = true)
-- S_F_Ratio: double (nullable = true)
-- perc_alumni: integer (nullable = true)
-- Expend: integer (nullable = true)
-- Grad_Rate: integer (nullable = true)
-- features: vector (nullable = true)
-- PrivateIndex: double (nullable = false)

In [14]:
final_data = output_fixed.select('features', 'PrivateIndex')

In [15]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [16]:
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, DecisionTreeClassifier

In [17]:
from pyspark.ml import Pipeline

In [18]:
dtc = DecisionTreeClassifier(labelCol='PrivateIndex', featuresCol='features')
rfc = RandomForestClassifier(numTrees=150, labelCol='PrivateIndex', featuresCol='features')
gbt = GBTClassifier(labelCol='PrivateIndex', featuresCol='features')

In [19]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [20]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [21]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [22]:
my_binary_eval = BinaryClassificationEvaluator(labelCol='PrivateIndex')

In [23]:
print('DTC')
print(my_binary_eval.evaluate(dtc_preds))
print('\n')

print('RFC')
print(my_binary_eval.evaluate(rfc_preds))
print('\n')

print('GBT')
print(my_binary_eval.evaluate(gbt_preds))

DTC
0.8775538250114521


RFC
0.9785616124599177


GBT
0.9390746678882272

In [24]:
my_binary_eval2 = BinaryClassificationEvaluator(labelCol='PrivateIndex', rawPredictionCol='prediction')

In [25]:
print('GBT')
print(my_binary_eval2.evaluate(gbt_preds))

GBT
0.8601007787448466

In [26]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [27]:
acc_eval = MulticlassClassificationEvaluator(metricName='accuracy', labelCol='PrivateIndex')

In [28]:
rfc_acc = acc_eval.evaluate(rfc_preds)
rfc_acc

Out[43]: 0.9221311475409836